<a href="https://colab.research.google.com/github/a2m-dotcom/DLBCL_Pub/blob/main/Mitotic%20figures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [1]:
!pip install numpy==1.26.4



In [3]:
!pip install slidesrunner openslide-python opencv-python shapely
!pip install torch torchvision


ERROR: Could not find a version that satisfies the requirement slidesrunner (from versions: none)
ERROR: No matching distribution found for slidesrunner
Fatal Python error: init_import_site: Failed to import the site module
Python runtime state: initialized
Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 1360, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1331, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 935, in _load_unlocked
  File "<frozen importlib._bootstrap>", line 1176, in exec_module
  File "<frozen site>", line 652, in <module>
  File "<frozen site>", line 639, in main
  File "<frozen site>", line 421, in addsitepackages
  File "<frozen site>", line 253, in addsitedir
  File "<frozen site>", line 212, in addpackage
  File "<string>", line 0, in <module>
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<frozen importlib._

In [2]:
import sqlite3
import pandas as pd

db_path = "/content/drive/MyDrive/MIDOGPP/MIDOGpp/images/MIDOGpp.sqlite"
conn = sqlite3.connect(db_path)

# Load tables
annotations = pd.read_sql_query("SELECT * FROM Annotations", conn)
coords      = pd.read_sql_query("SELECT * FROM Annotations_coordinates", conn)
labels      = pd.read_sql_query("SELECT * FROM Annotations_label", conn)
slides      = pd.read_sql_query("SELECT * FROM Slides", conn)

# Merge coordinates
df = annotations.merge(coords, left_on="uid", right_on="annoId", how="left")

# Merge labels
df = df.merge(labels[["annoId", "class"]], on="annoId", how="left")

# FIX: use 'slide_y' to merge with slide filenames
df = df.merge(slides[["uid", "filename"]], left_on="slide_y", right_on="uid", how="left")

# Clean column names
df_clean = df.rename(columns={
    "uid_x": "annotationID",
    "slide_y": "slideID",
    "coordinateX": "x",
    "coordinateY": "y"
})

df_clean = df_clean[["annotationID", "slideID", "x", "y", "class", "filename"]]

df_clean.head()


,annotationID,slideID,x,y,class,filename
0,1,1,4361.0,371.0,2,001.tiff
1,1,1,4361.0,371.0,2,001.tiff
2,2,1,781.0,897.0,2,001.tiff
3,2,1,781.0,897.0,2,001.tiff
4,3,1,295.0,4069.0,2,001.tiff


In [3]:
import os

crops_dir = "/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops"
masks_dir = "/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks"
images_dir = "/content/drive/MyDrive/MIDOGPP/MIDOGpp/images"

def find_file(base, id):
    """Find file by annotationID with allowed extensions."""
    for ext in [".png", ".jpg", ".jpeg"]:
        p = os.path.join(base, str(id) + ext)
        if os.path.exists(p):
            return p
    return None

df_clean["crop_path"] = df_clean["annotationID"].apply(lambda x: find_file(crops_dir, x))
df_clean["mask_path"] = df_clean["annotationID"].apply(lambda x: find_file(masks_dir, x))

# image comes from slide filename
df_clean["image_path"] = df_clean["filename"].apply(lambda fn: os.path.join(images_dir, fn))

df_final = df_clean.dropna(subset=["crop_path", "mask_path"]).reset_index(drop=True)

print("Total annotations:", len(df_clean))
print("Usable MF samples:", len(df_final))
df_final.head()

Total annotations: 62614
Usable MF samples: 62233


,annotationID,slideID,x,y,class,filename,crop_path,mask_path,image_path
0,1,1,4361.0,371.0,2,001.tiff,/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/1...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks/1...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/images/...
1,1,1,4361.0,371.0,2,001.tiff,/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/1...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks/1...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/images/...
2,2,1,781.0,897.0,2,001.tiff,/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/2...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks/2...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/images/...
3,2,1,781.0,897.0,2,001.tiff,/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/2...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks/2...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/images/...
4,3,1,295.0,4069.0,2,001.tiff,/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/3...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks/3...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/images/...


In [5]:
df_final = df_clean.drop_duplicates(subset=["annotationID"]).reset_index(drop=True)

print("Final unique samples:", len(df_final))
df_final.head(10)

Final unique samples: 26286


,annotationID,slideID,x,y,class,filename,crop_path,mask_path,image_path
0,1,1,4361.0,371.0,2,001.tiff,/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/1...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks/1...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/images/...
1,2,1,781.0,897.0,2,001.tiff,/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/2...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks/2...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/images/...
2,3,1,295.0,4069.0,2,001.tiff,/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/3...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks/3...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/images/...
3,4,1,6697.5,731.5,0,001.tiff,/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/4...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks/4...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/images/...
4,5,2,1897.0,344.0,2,002.tiff,/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/5...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks/5...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/images/...
5,6,2,4422.0,216.0,2,002.tiff,/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/6...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks/6...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/images/...
6,7,2,1867.0,1583.0,1,002.tiff,/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/7...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks/7...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/images/...
7,8,2,3808.0,2371.0,2,002.tiff,/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/8...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks/8...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/images/...
8,9,2,5344.0,3277.0,1,002.tiff,/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/9...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks/9...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/images/...
9,10,2,6267.0,3218.0,1,002.tiff,/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/1...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks/1...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/images/...


In [6]:
df_final["label"] = (df_final["class"] == 2).astype("float32")

In [7]:
df_clean = df_final.copy()

df_clean["crop_path"] = df_clean["crop_path"].astype(str)
df_clean["mask_path"] = df_clean["mask_path"].astype(str)

df_clean = df_clean[
    (~df_clean["crop_path"].str.contains("nan|None")) &
    (~df_clean["mask_path"].str.contains("nan|None"))
]

df_clean = df_clean[df_clean["crop_path"].apply(os.path.exists)]
df_clean = df_clean[df_clean["mask_path"].apply(os.path.exists)]

print("Remaining valid samples:", len(df_clean))

Remaining valid samples: 26129


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
train_df, test_df = train_test_split(
    df_clean, test_size=0.20, stratify=df_clean["label"], random_state=42
)
train_df, val_df = train_test_split(
    train_df, test_size=0.10, stratify=train_df["label"], random_state=42
)

In [10]:
import tensorflow as tf
train_paths  = tf.constant(train_df["crop_path"].tolist(), dtype=tf.string)
train_labels = tf.constant(train_df["label"].tolist(), dtype=tf.float32)

val_paths  = tf.constant(val_df["crop_path"].tolist(), dtype=tf.string)
val_labels = tf.constant(val_df["label"].tolist(), dtype=tf.float32)

test_paths  = tf.constant(test_df["crop_path"].tolist(), dtype=tf.string)
test_labels = tf.constant(test_df["label"].tolist(), dtype=tf.float32)

In [11]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 128
AUTO = tf.data.AUTOTUNE

def mask_from_crop(crop_path):
    return tf.strings.regex_replace(crop_path, "/crops/", "/masks/")

@tf.function
def load_fused(crop_path, label):
    mask_path = mask_from_crop(crop_path)

    crop = tf.io.read_file(crop_path)
    crop = tf.image.decode_png(crop, channels=3)
    crop = tf.image.convert_image_dtype(crop, tf.float32)
    crop = tf.image.resize(crop, IMG_SIZE)

    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)
    mask = tf.image.convert_image_dtype(mask, tf.float32)
    mask = tf.image.resize(mask, IMG_SIZE)

    fused = tf.concat([crop, mask], axis=-1)
    return fused, label

def make_dataset(paths, labels, augment=False):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(load_fused, num_parallel_calls=AUTO)
    ds = ds.batch(BATCH_SIZE).prefetch(AUTO)
    return ds

train_ds = make_dataset(train_paths, train_labels)
val_ds   = make_dataset(val_paths, val_labels)
test_ds  = make_dataset(test_paths, test_labels)

In [12]:
def build_resnet():
    inp = tf.keras.layers.Input(shape=(224, 224, 4))

    # ✅ Properly slice input
    rgb = tf.keras.layers.Lambda(lambda x: x[..., :3])(inp)
    mask = tf.keras.layers.Lambda(lambda x: x[..., 3:])(inp)

    # ✅ ResNet preprocessing
    rgb = tf.keras.applications.resnet.preprocess_input(rgb)

    # ✅ Load ResNet backbone
    base = tf.keras.applications.ResNet50(
        include_top=False,
        input_shape=(224, 224, 3),
        weights="imagenet"
    )

    x = base(rgb, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)

    # ✅ Mask branch
    m = tf.keras.layers.Conv2D(16, 3, activation="relu", padding="same")(mask)
    m = tf.keras.layers.GlobalAveragePooling2D()(m)

    # ✅ Fusion head
    z = tf.keras.layers.Concatenate()([x, m])
    z = tf.keras.layers.Dense(128, activation="relu")(z)
    z = tf.keras.layers.Dropout(0.3)(z)
    out = tf.keras.layers.Dense(1, activation="sigmoid")(z)

    return tf.keras.Model(inp, out)

In [13]:
model = build_resnet()
model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 4)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item (GetItem)  │ (None, 224, 224)  │          0 │ lambda[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_1          │ (None, 224, 224)  │          0 │ lambda[0][0]      │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_2          │ (None, 224, 224)  │          0 │ lambda[0][0]      │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack (Stack)       │ (None, 224, 224,  │          0 │ get_item[0][0],   │
│                     │ 3)                │            │ get_item_1[0][0], │
│                     │                   │            │ get_item_2[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 224, 224,  │          0 │ stack[0][0]       │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│                     │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resnet50            │ (None, 7, 7,      │ 23,587,712 │ add[0][0]         │
│ (Functional)        │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 224, 224,  │        160 │ lambda_1[0][0]    │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 2048)      │          0 │ resnet50[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 16)        │          0 │ conv2d[0][0]      │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 2064)      │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │    264,320 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │        129 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 23,852,321 (90.99 MB)

 Trainable params: 23,799,201 (90.79 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [14]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [16]:

SAVE_DIR = "/content/drive/MyDrive/MIDOGPP/MIDOGpp/models"
os.makedirs(SAVE_DIR, exist_ok=True)


In [17]:
ckpt_path = os.path.join(SAVE_DIR, "resnet50_best.h5")

In [18]:
ckpt = tf.keras.callbacks.ModelCheckpoint(
    ckpt_path,
    save_best_only=True,
    monitor="val_accuracy",
    mode="max",
    verbose=1
)

In [19]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=[ckpt]
)

Epoch 1/50
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.6383 - loss: 0.6391
Epoch 1: val_accuracy improved from -inf to 0.56930, saving model to /content/drive/MyDrive/MIDOGPP/MIDOGpp/models/resnet50_best.h5


166/166 ━━━━━━━━━━━━━━━━━━━━ 1663s 10s/step - accuracy: 0.6386 - loss: 0.6388 - val_accuracy: 0.5693 - val_loss: 0.8584
Epoch 2/50
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step - accuracy: 0.7796 - loss: 0.4692
Epoch 2: val_accuracy did not improve from 0.56930
166/166 ━━━━━━━━━━━━━━━━━━━━ 58s 350ms/step - accuracy: 0.7797 - loss: 0.4690 - val_accuracy: 0.5693 - val_loss: 0.8497
Epoch 3/50
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step - accuracy: 0.8386 - loss: 0.3622
Epoch 3: val_accuracy did not improve from 0.56930
166/166 ━━━━━━━━━━━━━━━━━━━━ 57s 341ms/step - accuracy: 0.8387 - loss: 0.3620 - val_accuracy: 0.5693 - val_loss: 0.9644
Epoch 4/50
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step - accuracy: 0.8815 - loss: 0.2710
Epoch 4: val_accuracy did not improve from 0.56930
166/166 ━━━━━━━━━━━━━━━━━━━━ 56s 339ms/step - accuracy: 0.8815 - loss: 0.2709 - val_accuracy: 0.5693 - val_loss: 1.3479
Epoch 5/50
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step - accuracy: 0.9077 - loss: 0.2225
Epoch 5: val_ac

166/166 ━━━━━━━━━━━━━━━━━━━━ 61s 369ms/step - accuracy: 0.9653 - loss: 0.0878 - val_accuracy: 0.5850 - val_loss: 0.7108
Epoch 9/50
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step - accuracy: 0.9742 - loss: 0.0680
Epoch 9: val_accuracy did not improve from 0.58503
166/166 ━━━━━━━━━━━━━━━━━━━━ 63s 374ms/step - accuracy: 0.9742 - loss: 0.0680 - val_accuracy: 0.5693 - val_loss: 3.7292
Epoch 10/50
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step - accuracy: 0.9767 - loss: 0.0638
Epoch 10: val_accuracy did not improve from 0.58503
166/166 ━━━━━━━━━━━━━━━━━━━━ 57s 340ms/step - accuracy: 0.9767 - loss: 0.0638 - val_accuracy: 0.5693 - val_loss: 3.3754
Epoch 11/50
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 303ms/step - accuracy: 0.9844 - loss: 0.0395
Epoch 11: val_accuracy did not improve from 0.58503
166/166 ━━━━━━━━━━━━━━━━━━━━ 56s 337ms/step - accuracy: 0.9844 - loss: 0.0395 - val_accuracy: 0.5693 - val_loss: 1.4354
Epoch 12/50
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 306ms/step - accuracy: 0.9868 - loss: 0.0352
Epoch 12: 

In [11]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_resnet():
    inp = layers.Input(shape=(224,224,4))

    # Split RGB + mask
    rgb  = inp[..., :3]
    mask = inp[..., 3:]

    base = tf.keras.applications.ResNet50(
        include_top=False,
        input_shape=(224,224,3),
        weights="imagenet"
    )

    x = base(rgb)
    x = layers.GlobalAveragePooling2D()(x)

    # Mask branch
    m = layers.Conv2D(16, 3, activation="relu", padding="same")(mask)
    m = layers.GlobalAveragePooling2D()(m)

    # Fusion
    z = layers.Concatenate()([x, m])
    z = layers.Dense(128, activation="relu")(z)
    z = layers.Dropout(0.3)(z)
    out = layers.Dense(1, activation="sigmoid")(z)

    return models.Model(inp, out)

In [13]:
model = build_resnet()

ckpt_path = "/content/drive/MyDrive/MIDOGPP/MIDOGpp/models/resnet50_best.h5"
model.load_weights(ckpt_path)

print("✅ ResNet weights loaded!")

✅ ResNet weights loaded!


In [14]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [15]:
loss, acc = model.evaluate(test_ds)

print("✅ ResNet Test Loss:", loss)
print("✅ ResNet Test Accuracy:", acc)

21/21 ━━━━━━━━━━━━━━━━━━━━ 95s 4s/step - accuracy: 0.4371 - loss: 110.7391
✅ ResNet Test Loss: 112.0219955444336
✅ ResNet Test Accuracy: 0.430539608001709


In [ ]:
def build_efficientnet():
    inp = tf.keras.layers.Input(shape=(224,224,4))
    rgb = inp[..., :3]
    mask = inp[..., 3:]

    base = tf.keras.applications.EfficientNetB0(
        include_top=False,
        input_shape=(224,224,3),
        weights="imagenet"
    )
    x = base(rgb)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)

    m = tf.keras.layers.Conv2D(16, 3, activation="relu", padding="same")(mask)
    m = tf.keras.layers.GlobalAveragePooling2D()(m)

    z = tf.keras.layers.Concatenate()([x, m])
    z = tf.keras.layers.Dense(128, activation="relu")(z)
    z = tf.keras.layers.Dropout(0.3)(z)
    out = tf.keras.layers.Dense(1, activation="sigmoid")(z)

    return tf.keras.Model(inp, out)

SAVE_DIR = "/content/drive/MyDrive/MIDOGPP/MIDOGpp/models"
os.makedirs(SAVE_DIR, exist_ok=True)

model = build_efficientnet()
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss="binary_crossentropy",
              metrics=["accuracy"])

ckpt_path = os.path.join(SAVE_DIR, "efficientnet_best(80-20).h5")

ckpt = tf.keras.callbacks.ModelCheckpoint(
    ckpt_path,
    save_best_only=True,
    monitor="val_accuracy",
    mode="max",
    verbose=1
)

history = model.fit(train_ds, validation_data=val_ds, epochs=50, callbacks=[ckpt])

Epoch 1/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.5812 - loss: 0.6682
Epoch 1: val_accuracy improved from -inf to 0.43089, saving model to /content/drive/MyDrive/MIDOGPP/MIDOGpp/models/efficientnet_best(80-20).h5


147/147 ━━━━━━━━━━━━━━━━━━━━ 1399s 9s/step - accuracy: 0.5816 - loss: 0.6679 - val_accuracy: 0.4309 - val_loss: 0.7039
Epoch 2/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 320ms/step - accuracy: 0.7690 - loss: 0.4858
Epoch 2: val_accuracy improved from 0.43089 to 0.55858, saving model to /content/drive/MyDrive/MIDOGPP/MIDOGpp/models/efficientnet_best(80-20).h5


147/147 ━━━━━━━━━━━━━━━━━━━━ 55s 373ms/step - accuracy: 0.7691 - loss: 0.4856 - val_accuracy: 0.5586 - val_loss: 0.6893
Epoch 3/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 325ms/step - accuracy: 0.8323 - loss: 0.3674
Epoch 3: val_accuracy did not improve from 0.55858
147/147 ━━━━━━━━━━━━━━━━━━━━ 53s 360ms/step - accuracy: 0.8324 - loss: 0.3673 - val_accuracy: 0.4902 - val_loss: 0.8422
Epoch 4/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step - accuracy: 0.8842 - loss: 0.2787
Epoch 4: val_accuracy did not improve from 0.55858
147/147 ━━━━━━━━━━━━━━━━━━━━ 52s 355ms/step - accuracy: 0.8842 - loss: 0.2786 - val_accuracy: 0.4821 - val_loss: 0.7517
Epoch 5/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step - accuracy: 0.9200 - loss: 0.2006
Epoch 5: val_accuracy did not improve from 0.55858
147/147 ━━━━━━━━━━━━━━━━━━━━ 52s 352ms/step - accuracy: 0.9200 - loss: 0.2005 - val_accuracy: 0.5414 - val_loss: 0.7243
Epoch 6/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step - accuracy: 0.9463 - loss: 0.1373
Epoch 6: val_ac

147/147 ━━━━━━━━━━━━━━━━━━━━ 55s 371ms/step - accuracy: 0.9632 - loss: 0.1011 - val_accuracy: 0.5662 - val_loss: 1.1004
Epoch 8/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 329ms/step - accuracy: 0.9692 - loss: 0.0785
Epoch 8: val_accuracy improved from 0.56624 to 0.59206, saving model to /content/drive/MyDrive/MIDOGPP/MIDOGpp/models/efficientnet_best(80-20).h5


147/147 ━━━━━━━━━━━━━━━━━━━━ 56s 383ms/step - accuracy: 0.9692 - loss: 0.0785 - val_accuracy: 0.5921 - val_loss: 1.4573
Epoch 9/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 326ms/step - accuracy: 0.9769 - loss: 0.0665
Epoch 9: val_accuracy did not improve from 0.59206
147/147 ━━━━━━━━━━━━━━━━━━━━ 53s 362ms/step - accuracy: 0.9769 - loss: 0.0665 - val_accuracy: 0.5519 - val_loss: 0.7028
Epoch 10/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 320ms/step - accuracy: 0.9797 - loss: 0.0558
Epoch 10: val_accuracy did not improve from 0.59206
147/147 ━━━━━━━━━━━━━━━━━━━━ 53s 357ms/step - accuracy: 0.9797 - loss: 0.0558 - val_accuracy: 0.5696 - val_loss: 1.3018
Epoch 11/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step - accuracy: 0.9830 - loss: 0.0474
Epoch 11: val_accuracy did not improve from 0.59206
147/147 ━━━━━━━━━━━━━━━━━━━━ 52s 354ms/step - accuracy: 0.9830 - loss: 0.0474 - val_accuracy: 0.5825 - val_loss: 1.7997
Epoch 12/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 325ms/step - accuracy: 0.9840 - loss: 0.0470
Epoch 12: 

147/147 ━━━━━━━━━━━━━━━━━━━━ 55s 376ms/step - accuracy: 0.9870 - loss: 0.0391 - val_accuracy: 0.6887 - val_loss: 1.4545
Epoch 15/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step - accuracy: 0.9886 - loss: 0.0325
Epoch 15: val_accuracy did not improve from 0.68867
147/147 ━━━━━━━━━━━━━━━━━━━━ 54s 363ms/step - accuracy: 0.9886 - loss: 0.0325 - val_accuracy: 0.5691 - val_loss: 1.4209
Epoch 16/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step - accuracy: 0.9897 - loss: 0.0326
Epoch 16: val_accuracy did not improve from 0.68867
147/147 ━━━━━━━━━━━━━━━━━━━━ 52s 354ms/step - accuracy: 0.9897 - loss: 0.0326 - val_accuracy: 0.5677 - val_loss: 1.4196
Epoch 17/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step - accuracy: 0.9898 - loss: 0.0302
Epoch 17: val_accuracy did not improve from 0.68867
147/147 ━━━━━━━━━━━━━━━━━━━━ 52s 355ms/step - accuracy: 0.9898 - loss: 0.0302 - val_accuracy: 0.5610 - val_loss: 1.1328
Epoch 18/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step - accuracy: 0.9896 - loss: 0.0273
Epoch 18

In [13]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_efficientnet():
    inp = layers.Input(shape=(224,224,4))

    rgb  = inp[..., :3]
    mask = inp[..., 3:]

    base = tf.keras.applications.EfficientNetB0(
        include_top=False,
        input_shape=(224,224,3),
        weights="imagenet"
    )

    x = base(rgb)
    x = layers.GlobalAveragePooling2D()(x)

    m = layers.Conv2D(16, 3, activation="relu", padding="same")(mask)
    m = layers.GlobalAveragePooling2D()(m)

    z = layers.Concatenate()([x, m])
    z = layers.Dense(128, activation="relu")(z)
    z = layers.Dropout(0.3)(z)
    out = layers.Dense(1, activation="sigmoid")(z)

    return models.Model(inp, out)

In [14]:
model = build_efficientnet()

ckpt_path = "/content/drive/MyDrive/MIDOGPP/MIDOGpp/models/efficientnet_best(80-20).h5"
model.load_weights(ckpt_path)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

loss, acc = model.evaluate(test_ds)
print("✅ EfficientNet Test Accuracy:", acc)
print("✅ EfficientNet Test Loss:", loss)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
41/41 ━━━━━━━━━━━━━━━━━━━━ 307s 7s/step - accuracy: 0.6793 - loss: 1.4991
✅ EfficientNet Test Accuracy: 0.6836969256401062
✅ EfficientNet Test Loss: 1.4705231189727783


In [15]:
# Get predictions
y_probs = model.predict(test_ds)
y_pred  = (y_probs > 0.5).astype(int).flatten()

# Get true labels
import numpy as np
y_true = np.concatenate([y.numpy() for _, y in test_ds])

# Metrics
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, classification_report

precision = precision_score(y_true, y_pred)
recall    = recall_score(y_true, y_pred)
f1        = f1_score(y_true, y_pred)
auc       = roc_auc_score(y_true, y_probs)

print("\n✅ Precision:", precision)
print("✅ Recall:", recall)
print("✅ F1-score:", f1)
print("✅ ROC-AUC:", auc)

print("\n✅ Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

print("\n✅ Full Classification Report:")
print(classification_report(y_true, y_pred))

41/41 ━━━━━━━━━━━━━━━━━━━━ 26s 459ms/step

✅ Precision: 0.6638593014614813
✅ Recall: 0.9005376344086021
✅ F1-score: 0.7642948809354057
✅ ROC-AUC: 0.7248302718040621

✅ Confusion Matrix:
[[ 893 1357]
 [ 296 2680]]

✅ Full Classification Report:
              precision    recall  f1-score   support

         0.0       0.75      0.40      0.52      2250
         1.0       0.66      0.90      0.76      2976

    accuracy                           0.68      5226
   macro avg       0.71      0.65      0.64      5226
weighted avg       0.70      0.68      0.66      5226



In [15]:
model = build_efficientnet()
model.load_weights(ckpt_path)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss="binary_crossentropy",
              metrics=["accuracy"])

loss, acc = model.evaluate(test_ds)
print("EfficientNet Test Loss:", loss)
print("EfficientNet Test Accuracy:", acc)

21/21 ━━━━━━━━━━━━━━━━━━━━ 310s 14s/step - accuracy: 0.7796 - loss: 1.1108
EfficientNet Test Loss: 1.152380347251892
EfficientNet Test Accuracy: 0.772675096988678


In [7]:
print(df_final["label"].value_counts())

label
1.0    14961
0.0    11325
Name: count, dtype: int64


In [8]:

import tensorflow as tf
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

In [9]:
df_used = df_final.copy()
df_used["label"] = (df_used["class"] == 2).astype("float32")

In [10]:
df_cleaned = df_final.copy()

# ensure everything is string
df_cleaned["crop_path"] = df_cleaned["crop_path"].astype(str)
df_cleaned["mask_path"] = df_cleaned["mask_path"].astype(str)

# remove rows with 'nan', 'None', or 'NaN'
df_cleaned = df_cleaned[
    (df_cleaned["crop_path"].str.contains("nan") == False) &
    (df_cleaned["crop_path"].str.contains("None") == False) &
    (df_cleaned["mask_path"].str.contains("nan") == False) &
    (df_cleaned["mask_path"].str.contains("None") == False)
]

import os
df_cleaned = df_cleaned[df_cleaned["crop_path"].apply(os.path.exists)]
df_cleaned = df_cleaned[df_cleaned["mask_path"].apply(os.path.exists)]

In [18]:
df_cleaned.head()
len(df_cleaned)

26129

In [11]:
train_df, test_df = train_test_split(df_cleaned, test_size=0.10, stratify=df_cleaned["label"], random_state=42)
train_df, val_df  = train_test_split(train_df, test_size=0.10, stratify=train_df["label"], random_state=42)

In [12]:
train_paths = tf.constant(train_df["crop_path"].tolist())
train_labels = tf.constant(train_df["label"].tolist(), dtype=tf.float32)

val_paths = tf.constant(val_df["crop_path"].tolist())
val_labels = tf.constant(val_df["label"].tolist(), dtype=tf.float32)

test_paths = tf.constant(test_df["crop_path"].tolist())
test_labels = tf.constant(test_df["label"].tolist(), dtype=tf.float32)

In [13]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 128
AUTO = tf.data.AUTOTUNE

def mask_from_crop(crop_path):
    return tf.strings.regex_replace(crop_path, "/crops/", "/masks/")

@tf.function
def load_fused(crop_path, label):
    mask_path = mask_from_crop(crop_path)

    crop = tf.io.read_file(crop_path)
    crop = tf.image.decode_png(crop, channels=3)
    crop = tf.image.convert_image_dtype(crop, tf.float32)
    crop = tf.image.resize(crop, IMG_SIZE)

    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)
    mask = tf.image.convert_image_dtype(mask, tf.float32)
    mask = tf.image.resize(mask, IMG_SIZE)

    fused = tf.concat([crop, mask], axis=-1)
    return fused, label

@tf.function
def augment(fused, label):
    img = fused[..., :3]
    msk = fused[..., 3:]

    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_flip_up_down(img)
    img = tf.image.random_brightness(img, 0.1)
    img = tf.image.random_contrast(img, 0.9, 1.1)

    fused = tf.concat([img, msk], axis=-1)
    return fused, label

In [14]:
def make_dataset(paths, labels, augment_flag=False):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))

    if augment_flag:
        ds = ds.shuffle(5000, seed=42)

    ds = ds.map(load_fused, num_parallel_calls=AUTO)

    if augment_flag:
        ds = ds.map(augment, num_parallel_calls=AUTO)

    # Cache results for extremely fast epoch 2+
    ds = ds.cache("/content/cache_mitotic.tfcache")

    ds = ds.batch(BATCH_SIZE).prefetch(AUTO)
    return ds

In [15]:
train_ds = make_dataset(train_paths, train_labels, augment_flag=True)
val_ds   = make_dataset(val_paths, val_labels, augment_flag=False)
test_ds  = make_dataset(test_paths, test_labels, augment_flag=False)

# sanity check
for X, y in train_ds.take(1):
    print(X.shape, y.shape)

(128, 224, 224, 4) (128,)


In [24]:
def build_mobilenet():
    inp = layers.Input(shape=(224,224,4))
    rgb = inp[..., :3]
    mask = inp[..., 3:]

    base = tf.keras.applications.MobileNetV2(input_shape=(224,224,3), include_top=False, weights="imagenet")
    x = base(rgb)
    x = layers.GlobalAveragePooling2D()(x)

    m = layers.Conv2D(16, 3, activation="relu", padding="same")(mask)
    m = layers.GlobalAveragePooling2D()(m)

    z = layers.Concatenate()([x, m])
    z = layers.Dense(128, activation="relu")(z)
    z = layers.Dropout(0.3)(z)
    out = layers.Dense(1, activation="sigmoid")(z)
    return models.Model(inp, out)

In [27]:
!rm -f /content/cache_mitotic.tfcache*

In [16]:
train_ds = make_dataset(train_paths, train_labels, augment_flag=True)
val_ds   = make_dataset(val_paths, val_labels)
test_ds  = make_dataset(test_paths, test_labels)

In [17]:
for X, y in train_ds.take(1):
    print(X.shape, y.shape)
    break

(128, 224, 224, 4) (128,)


In [28]:
SAVE_DIR = "/content/drive/MyDrive/MIDOGPP/MIDOGpp/models"
os.makedirs(SAVE_DIR, exist_ok=True)

model = build_mobilenet()
# model = build_efficientnet()
# model = build_resnet()

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

ckpt_path = os.path.join(SAVE_DIR, "mobilenet_best.h5")

ckpt = tf.keras.callbacks.ModelCheckpoint(
    ckpt_path,
    save_best_only=True,
    monitor="val_accuracy",
    mode="max",
    verbose=1
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=[ckpt]
)

Epoch 1/50


Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x77fe06926480>
Traceback (most recent call last):
  File "/usr/lib/python3.12/weakref.py", line 369, in remove
    def remove(k, selfref=ref(self)):

KeyboardInterrupt: 


KeyboardInterrupt: 

In [7]:
import tensorflow as tf

IMG_SIZE = (224, 224)

def load_and_resize(path, channels):
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img, channels=channels, expand_animations=False)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, IMG_SIZE)
    return img

def fusion_loader(crop_path, mask_path, label):
    crop = load_and_resize(crop_path, 3)
    mask = load_and_resize(mask_path, 1)
    fused = tf.concat([crop, mask], axis=-1)   # (224, 224, 4)
    label = tf.cast(label, tf.float32)
    return fused, label

def make_dataset(df, batch=32, shuffle=False):
    ds = tf.data.Dataset.from_tensor_slices(
        (df["crop_path"], df["mask_path"], df["class"])
    )
    if shuffle:
        ds = ds.shuffle(len(df))
    ds = ds.map(fusion_loader, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch).prefetch(tf.data.AUTOTUNE)
    return ds

In [8]:
import tensorflow as tf

def augment(fused, label):
    # Random flips
    fused = tf.image.random_flip_left_right(fused)
    fused = tf.image.random_flip_up_down(fused)

    # Random rotation (0, 90, 180, 270 degrees)
    k = tf.random.uniform(shape=(), minval=0, maxval=4, dtype=tf.int32)
    fused = tf.image.rot90(fused, k)

    # Random brightness for RGB channels (only apply to crop, not mask)
    rgb = fused[..., :3]
    mask = fused[..., 3:]

    rgb = tf.image.random_brightness(rgb, max_delta=0.15)
    rgb = tf.image.random_contrast(rgb, lower=0.8, upper=1.2)

    # Optional slight Gaussian noise
    noise = tf.random.normal(tf.shape(rgb), mean=0.0, stddev=0.03)
    rgb = tf.clip_by_value(rgb + noise, 0.0, 1.0)

    fused = tf.concat([rgb, mask], axis=-1)
    return fused, label

In [9]:
def make_dataset(df, batch=32, shuffle=False, augment_data=False):
    ds = tf.data.Dataset.from_tensor_slices(
        (df["crop_path"], df["mask_path"], df["class"])
    )

    if shuffle:
        ds = ds.shuffle(len(df))

    ds = ds.map(fusion_loader, num_parallel_calls=tf.data.AUTOTUNE)

    if augment_data:
        ds = ds.map(augment, num_parallel_calls=tf.data.AUTOTUNE)

    ds = ds.batch(batch).prefetch(tf.data.AUTOTUNE)
    return ds

In [18]:
# Requirements: pandas, sklearn
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split, GroupShuffleSplit

# df_final should be the dataframe you built with columns:
# ['annotationID','slideID','x','y','class','filename','crop_path','mask_path','image_path']

# 1) Deduplicate by annotationID (keep first)
df = df_final.drop_duplicates(subset=["annotationID"]).reset_index(drop=True)

# 2) Drop rows with any missing paths (this prevents NoneType errors)
df = df.dropna(subset=["crop_path", "mask_path", "image_path"]).reset_index(drop=True)

# 3) Ensure all files actually exist on disk (optional but recommended)
def file_exists(p):
    return (isinstance(p, str) and os.path.exists(p))
exists_mask = df["crop_path"].apply(file_exists) & df["mask_path"].apply(file_exists) & df["image_path"].apply(file_exists)
df = df[exists_mask].reset_index(drop=True)

print("After dedupe & file-check: total samples =", len(df))

# 4) Optional: convert class to int (0/1)
df['class'] = df['class'].astype(int)  # or map your labels to 0/1

# 5) Group-aware split by slideID to avoid leakage.
#    We first split off the test set (e.g. 10%) by grouping on slideID
gss = GroupShuffleSplit(n_splits=1, test_size=0.10, random_state=42)
train_idx, test_idx = next(gss.split(df, groups=df['slideID']))
df_temp = df.iloc[train_idx].reset_index(drop=True)
df_test = df.iloc[test_idx].reset_index(drop=True)

# Then split train -> train+val (e.g., 10% of remaining for val)
gss2 = GroupShuffleSplit(n_splits=1, test_size=0.10, random_state=42)
train_idx2, val_idx2 = next(gss2.split(df_temp, groups=df_temp['slideID']))
df_train = df_temp.iloc[train_idx2].reset_index(drop=True)
df_val   = df_temp.iloc[val_idx2].reset_index(drop=True)

print("Train/Val/Test sizes:", len(df_train), len(df_val), len(df_test))

# 6) Create tf.data datasets from plain lists (no Series with None)
IMG_SIZE = (224,224)

def load_and_resize(path, channels):
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img, channels=channels, expand_animations=False)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, IMG_SIZE)
    return img

def fusion_loader(crop_path, mask_path, label):
    crop = load_and_resize(crop_path, 3)
    mask = load_and_resize(mask_path, 1)
    fused = tf.concat([crop, mask], axis=-1)   # (H,W,4)
    return fused, tf.cast(label, tf.float32)

def make_dataset_from_df(df_in, batch=32, shuffle=False, augment_fn=None):
    # use plain python lists (or numpy arrays)
    crops = df_in['crop_path'].tolist()
    masks = df_in['mask_path'].tolist()
    labels = df_in['class'].tolist()
    ds = tf.data.Dataset.from_tensor_slices((crops, masks, labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(crops), seed=42)
    ds = ds.map(lambda a,b,c: tf.py_function(
                    func=lambda x,y,z: fusion_loader(x.numpy().decode(), y.numpy().decode(), int(z.numpy())),
                    inp=[a,b,c],
                    Tout=[tf.float32, tf.float32]),
                num_parallel_calls=tf.data.AUTOTUNE)
    # fix static shapes (py_function loses shape)
    def set_shapes(fused, lab):
        fused.set_shape((IMG_SIZE[0], IMG_SIZE[1], 4))
        lab.set_shape(())
        return fused, lab
    ds = ds.map(set_shapes, num_parallel_calls=tf.data.AUTOTUNE)
    # optional augmentation
    if augment_fn is not None:
        ds = ds.map(lambda x,y: augment_fn(x,y), num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch).prefetch(tf.data.AUTOTUNE)
    return ds

# Example augmentation function (simple)
def augment(fused, label):
    rgb = fused[..., :3]
    mask = fused[..., 3:]
    # flips
    if tf.random.uniform(()) > 0.5:
        rgb = tf.image.flip_left_right(rgb); mask = tf.image.flip_left_right(mask)
    if tf.random.uniform(()) > 0.5:
        rgb = tf.image.flip_up_down(rgb); mask = tf.image.flip_up_down(mask)
    # small rotation
    k = tf.random.uniform(shape=(), minval=0, maxval=4, dtype=tf.int32)
    rgb = tf.image.rot90(rgb, k); mask = tf.image.rot90(mask, k)
    # brightness/contrast
    rgb = tf.image.random_brightness(rgb, 0.1)
    rgb = tf.image.random_contrast(rgb, 0.9, 1.1)
    fused = tf.concat([rgb, mask], axis=-1)
    return fused, label

# build datasets
train_ds = make_dataset_from_df(df_train, batch=32, shuffle=True, augment_fn=augment)
val_ds   = make_dataset_from_df(df_val, batch=32, shuffle=False, augment_fn=None)
test_ds  = make_dataset_from_df(df_test, batch=32, shuffle=False, augment_fn=None)

After dedupe & file-check: total samples = 26129
Train/Val/Test sizes: 22401 1855 1873


In [12]:
from tensorflow.keras import layers, Model
import tensorflow as tf

def build_mobilenet_fusion(input_shape=(224,224,4)):
    inp = layers.Input(shape=input_shape)

    # Split inputs
    rgb  = inp[..., :3]     # crop image
    mask = inp[..., 3:]     # binary SAM mask

    # ---------------------------
    # Image encoder (MobileNetV2)
    # ---------------------------
    base = tf.keras.applications.MobileNetV2(
        input_shape=(224,224,3),
        include_top=False,
        weights="imagenet"
    )
    x = base(rgb)
    x = layers.GlobalAveragePooling2D()(x)

    # ---------------------------
    # Mask encoder (small CNN)
    # ---------------------------
    m = layers.Conv2D(16, 3, padding="same", activation="relu")(mask)
    m = layers.GlobalAveragePooling2D()(m)

    # ---------------------------
    # Fusion + classifier head
    # ---------------------------
    z = layers.Concatenate()([x, m])
    z = layers.Dense(128, activation="relu")(z)
    z = layers.Dropout(0.3)(z)
    out = layers.Dense(1, activation="sigmoid")(z)

    return Model(inp, out)

model = build_mobilenet_fusion()
model.summary()

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 4)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item (GetItem)  │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_1          │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (GetItem)           │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mobilenetv2_1.00_2… │ (None, 7, 7,      │  2,257,984 │ get_item[0][0]    │
│ (Functional)        │ 1280)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 224, 224,  │        160 │ get_item_1[0][0]  │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 1280)      │          0 │ mobilenetv2_1.00… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 16)        │          0 │ conv2d[0][0]      │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 1296)      │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │    166,016 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │        129 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,424,289 (9.25 MB)

 Trainable params: 2,390,177 (9.12 MB)

 Non-trainable params: 34,112 (133.25 KB)

In [13]:
ckpt_path = "/content/mobilenet_fusion_best.h5"

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        ckpt_path,
        save_best_only=True,
        monitor="val_accuracy",
        mode="max"
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=4,
        verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=8,
        restore_best_weights=True
    )
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=callbacks
)

Epoch 1/50
 44/701 ━━━━━━━━━━━━━━━━━━━━ 38:48 4s/step - accuracy: 0.3988 - loss: -2.1831

KeyboardInterrupt: 

In [14]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

In [30]:
IMG_SIZE = (224, 224)   # Faster than 224, still good accuracy
BATCH_SIZE = 64         # Double speed if GPU allows

In [31]:
IMG_SIZE = (224,224)

def load_and_resize_tf(path, channels):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels=channels)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, IMG_SIZE)   # ← MUST USE THIS
    return img

In [28]:
def fusion_loader_tf(crop_path, mask_path, label):
    crop = load_and_resize_tf(crop_path, 3)
    mask = load_and_resize_tf(mask_path, 1)
    fused = tf.concat([crop, mask], axis=-1)
    return fused, tf.cast(label, tf.float32)

In [16]:
import tensorflow as tf

def load_and_resize_tf(path, channels):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels=channels)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, IMG_SIZE)
    return img

def fusion_loader_tf(crop_path, mask_path, label):
    crop = load_and_resize_tf(crop_path, 3)
    mask = load_and_resize_tf(mask_path, 1)
    fused = tf.concat([crop, mask], axis=-1)
    return fused, tf.cast(label, tf.float32)

In [17]:
def augment_fast(fused, label):
    rgb  = fused[..., :3]
    mask = fused[..., 3:]

    # Fast flips only
    rgb  = tf.image.random_flip_left_right(rgb)
    mask = tf.image.random_flip_left_right(mask)
    rgb  = tf.image.random_flip_up_down(rgb)
    mask = tf.image.random_flip_up_down(mask)

    # Light jitter
    rgb = tf.image.random_brightness(rgb, max_delta=0.10)
    rgb = tf.image.random_contrast(rgb, 0.9, 1.1)

    fused = tf.concat([rgb, mask], axis=-1)
    return fused, label

In [18]:
def make_dataset_fast(df, shuffle=False, augment=False):
    crops  = df["crop_path"].tolist()
    masks  = df["mask_path"].tolist()
    labels = df["class"].astype('float32').tolist()

    ds = tf.data.Dataset.from_tensor_slices((crops, masks, labels))

    if shuffle:
        ds = ds.shuffle(len(df), reshuffle_each_iteration=True)

    ds = ds.map(fusion_loader_tf, num_parallel_calls=tf.data.AUTOTUNE)

    if augment:
        ds = ds.map(augment_fast, num_parallel_calls=tf.data.AUTOTUNE)

    ds = ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return ds

In [20]:
import numpy as np
import os

df = df_final.copy()

# Fix annotationID being float → convert to int → string
df["annotationID"] = df["annotationID"].astype(int).astype(str)

# Force crop/mask paths into PURE strings
df["crop_path"] = df["crop_path"].astype(str)
df["mask_path"] = df["mask_path"].astype(str)
df["image_path"] = df["image_path"].astype(str)

# Remove rows containing 'nan' or 'None' in paths
df = df[
    (~df["crop_path"].str.contains("nan")) &
    (~df["mask_path"].str.contains("nan")) &
    (~df["image_path"].str.contains("nan"))
]

# Remove rows where file does not exist
df = df[df["crop_path"].apply(os.path.exists)]
df = df[df["mask_path"].apply(os.path.exists)]
df = df[df["image_path"].apply(os.path.exists)]

# MOST IMPORTANT: Deduplicate annotationID
df = df.drop_duplicates(subset="annotationID").reset_index(drop=True)

print("Cleaned rows:", len(df))
df.head()

Cleaned rows: 26129


,annotationID,slideID,x,y,class,filename,crop_path,mask_path,image_path
0,1,1,4361.0,371.0,2,001.tiff,/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/1...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks/1...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/images/...
1,2,1,781.0,897.0,2,001.tiff,/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/2...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks/2...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/images/...
2,3,1,295.0,4069.0,2,001.tiff,/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/3...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks/3...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/images/...
3,4,1,6697.5,731.5,0,001.tiff,/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/4...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks/4...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/images/...
4,5,2,1897.0,344.0,2,002.tiff,/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/5...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks/5...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/images/...


In [21]:
from tensorflow.keras import layers, Model

def build_mobilenet_fusion(input_shape=(160,160,4)):
    inp = layers.Input(shape=input_shape)

    rgb  = inp[..., :3]
    mask = inp[..., 3:]

    base = tf.keras.applications.MobileNetV2(
        input_shape=(160,160,3),
        include_top=False,
        weights="imagenet",
        alpha=0.35   # lighter model = faster
    )

    x = base(rgb)
    x = layers.GlobalAveragePooling2D()(x)

    m = layers.Conv2D(16, 3, padding="same", activation="relu")(mask)
    m = layers.GlobalAveragePooling2D()(m)

    z = layers.Concatenate()([x, m])
    z = layers.Dense(128, activation="relu")(z)
    z = layers.Dropout(0.3)(z)

    out = layers.Dense(1, activation="sigmoid", dtype='float32')(z)

    return Model(inp, out)

model = build_mobilenet_fusion()

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

2019640/2019640 ━━━━━━━━━━━━━━━━━━━━ 1s 1us/step


In [22]:
save_dir = "/content/drive/MyDrive/MIDOGPP/MIDOGpp/models"
os.makedirs(save_dir, exist_ok=True)

ckpt_path = f"{save_dir}/mobilenet_fast_best.h5"

In [26]:
tf.image.resize(img, IMG_SIZE)

NameError: name 'img' is not defined

In [34]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 64

In [35]:
def load_and_resize_tf(path, channels):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels=channels)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, IMG_SIZE)   # <-- 224x224 here
    return img

def fusion_loader_tf(crop_path, mask_path, label):
    crop = load_and_resize_tf(crop_path, 3)
    mask = load_and_resize_tf(mask_path, 1)
    fused = tf.concat([crop, mask], axis=-1)  # 4 channels
    return fused, tf.cast(label, tf.float32)

In [36]:
def augment_fast(fused, label):
    rgb  = fused[..., :3]
    mask = fused[..., 3:]

    # Flips
    rgb = tf.image.random_flip_left_right(rgb)
    mask = tf.image.random_flip_left_right(mask)

    rgb = tf.image.random_flip_up_down(rgb)
    mask = tf.image.random_flip_up_down(mask)

    # Light color jitter
    rgb = tf.image.random_brightness(rgb, 0.1)
    rgb = tf.image.random_contrast(rgb, 0.9, 1.1)

    fused = tf.concat([rgb, mask], axis=-1)
    return fused, label

In [37]:
def make_dataset_fast(df, shuffle=False, augment=False):
    crops  = df["crop_path"].tolist()
    masks  = df["mask_path"].tolist()
    labels = df["class"].astype('float32').tolist()

    ds = tf.data.Dataset.from_tensor_slices((crops, masks, labels))

    if shuffle:
        ds = ds.shuffle(len(df), seed=42, reshuffle_each_iteration=True)

    ds = ds.map(fusion_loader_tf, num_parallel_calls=tf.data.AUTOTUNE)

    if augment:
        ds = ds.map(augment_fast, num_parallel_calls=tf.data.AUTOTUNE)

    ds = ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return ds

In [40]:
IMG_SIZE = (224, 224)

def load_and_resize_tf(path, channels):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels=channels)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, IMG_SIZE)     # ALWAYS correct size
    return img

def fusion_loader_tf(crop_path, mask_path, label):
    crop = load_and_resize_tf(crop_path, 3)
    mask = load_and_resize_tf(mask_path, 1)
    fused = tf.concat([crop, mask], axis=-1) # ALWAYS shape (224,224,4)
    return fused, tf.cast(label, tf.float32)

In [41]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

from tensorflow.keras import layers, Model

def build_mobilenet_fusion(input_shape=(224,224,4)):
    inp = layers.Input(shape=input_shape)

    rgb  = inp[..., :3]
    mask = inp[..., 3:]

    base = tf.keras.applications.MobileNetV2(
        input_shape=(224,224,3),
        include_top=False,
        weights="imagenet",
        alpha=0.35
    )
    x = base(rgb)
    x = layers.GlobalAveragePooling2D()(x)

    m = layers.Conv2D(16, 3, padding="same", activation="relu")(mask)
    m = layers.GlobalAveragePooling2D()(m)

    z = layers.Concatenate()([x, m])
    z = layers.Dense(128, activation="relu")(z)
    z = layers.Dropout(0.3)(z)

    out = layers.Dense(1, activation="sigmoid", dtype='float32')(z)

    return Model(inp, out)

model = build_mobilenet_fusion()

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [42]:
save_dir = "/content/drive/MyDrive/MIDOGPP/MIDOGpp/models"
os.makedirs(save_dir, exist_ok=True)

ckpt_path = f"{save_dir}/mobilenet224_best.h5"

In [43]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        ckpt_path,
        save_best_only=True,
        monitor="val_accuracy",
        mode="max",
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=3,
        verbose=1
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=6,
        restore_best_weights=True,
        verbose=1
    )
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=callbacks
)

Epoch 1/50
 57/701 ━━━━━━━━━━━━━━━━━━━━ 36:30 3s/step - accuracy: 0.4115 - loss: -2.6054

KeyboardInterrupt: 

In [20]:
train_ds = make_dataset(df_train, shuffle=True, augment_data=True)
val_ds   = make_dataset(df_val, augment_data=False)
test_ds  = make_dataset(df_test, augment_data=False)

NameError: name 'df_train' is not defined

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df_final, test_size=0.10, random_state=42)
df_train, df_val  = train_test_split(df_train, test_size=0.10, random_state=42)

train_ds = make_dataset(df_train, shuffle=True, batch=32)
val_ds   = make_dataset(df_val, batch=32)
test_ds  = make_dataset(df_test, batch=32)

len(df_train), len(df_val), len(df_test)

In [2]:
df = df_clean.copy()  # from previous step

images_dir = "/content/drive/MyDrive/MIDOGPP/MIDOGpp/images"
crops_dir  = "/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops"


In [3]:
import os
os.makedirs(crops_dir, exist_ok=True)


In [4]:
import cv2
import numpy as np
from tqdm import tqdm

CROP_SIZE = 128
HALF = CROP_SIZE // 2

def safe_crop(img, x, y, crop_size=128):
    h, w = img.shape[:2]

    x1 = max(0, x - HALF)
    y1 = max(0, y - HALF)
    x2 = min(w, x + HALF)
    y2 = min(h, y + HALF)

    crop = img[y1:y2, x1:x2]

    # If crop is smaller at boundaries → pad to 128px
    if crop.shape[0] != CROP_SIZE or crop.shape[1] != CROP_SIZE:
        pad_y = CROP_SIZE - crop.shape[0]
        pad_x = CROP_SIZE - crop.shape[1]
        crop = np.pad(crop, ((0,pad_y),(0,pad_x),(0,0)), mode='constant', constant_values=0)

    return crop


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

CROP_SIZE = 128
HALF = CROP_SIZE // 2

def safe_crop(img, x, y):
    h, w = img.shape[:2]

    x1 = max(0, x - HALF)
    y1 = max(0, y - HALF)
    x2 = min(w, x + HALF)
    y2 = min(h, y + HALF)

    crop = img[y1:y2, x1:x2]

    if crop.shape[0] != CROP_SIZE or crop.shape[1] != CROP_SIZE:
        pad_y = CROP_SIZE - crop.shape[0]
        pad_x = CROP_SIZE - crop.shape[1]
        crop = np.pad(crop, ((0, pad_y), (0, pad_x), (0, 0)),
                      mode='constant', constant_values=0)

    return crop


images_dir = "/content/drive/MyDrive/MIDOGPP/MIDOGpp/images"
crops_dir  = "/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops"
os.makedirs(crops_dir, exist_ok=True)

# GROUP BY SLIDE
grouped = df.groupby("filename")

for slide_name, group in tqdm(grouped, total=len(grouped)):
    slide_path = f"{images_dir}/{slide_name}"

    img = cv2.imread(slide_path)
    if img is None:
        print("Could not open:", slide_path)
        continue

    # Process all annotations belonging to THIS slide
    for _, row in group.iterrows():
        x = int(row["x"])
        y = int(row["y"])
        ann = int(row["annotationID"])

        crop = safe_crop(img, x, y)
        crop_path = f"{crops_dir}/{ann}.png"
        cv2.imwrite(crop_path, crop)



  6%|▌         | 30/503 [01:38<18:54,  2.40s/it]

Could not open: /content/drive/MyDrive/MIDOGPP/MIDOGpp/images/030.tiff


  7%|▋         | 34/503 [01:45<13:21,  1.71s/it]

Could not open: /content/drive/MyDrive/MIDOGPP/MIDOGpp/images/034.tiff


 63%|██████▎   | 315/503 [17:03<06:56,  2.22s/it]

Could not open: /content/drive/MyDrive/MIDOGPP/MIDOGpp/images/365.tiff


100%|██████████| 503/503 [27:26<00:00,  3.27s/it]


In [ ]:
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install opencv-python matplotlib


  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-787nk8ft
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-787nk8ft
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 76, in resolve
    collected = self.factory.collect_root_requirements(root_reqs)
          

In [ ]:
!wget -O sam_vit_h.pth https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth


--2025-11-30 19:49:08--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.173.132.81, 18.173.132.13, 18.173.132.6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.173.132.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2564550879 (2.4G) [binary/octet-stream]
Saving to: ‘sam_vit_h.pth’

sam_vit_h.pth       100%[===================>]   2.39G   133MB/s    in 13s     

2025-11-30 19:49:21 (187 MB/s) - ‘sam_vit_h.pth’ saved [2564550879/2564550879]



In [ ]:
import torch
from segment_anything import sam_model_registry, SamPredictor

sam_checkpoint = "sam_vit_h.pth"
model_type = "vit_h"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to("cuda")
predictor = SamPredictor(sam)


ModuleNotFoundError: No module named 'segment_anything'

In [ ]:
import numpy as np
import cv2
import os

def get_sam_mask(crop):
    crop_rgb = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
    predictor.set_image(crop_rgb)

    input_point = np.array([[64, 64]])   # center point
    input_label = np.array([1])          # foreground

    masks, scores, _ = predictor.predict(
        point_coords=input_point,
        point_labels=input_label,
        multimask_output=False,
    )

    mask = masks[0].astype(np.uint8) * 255
    return mask


In [ ]:
masks_dir = "/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks"
os.makedirs(masks_dir, exist_ok=True)


In [ ]:
df_clean["crop_path"] = df_clean["annotationID"].apply(
    lambda x: f"{crops_dir}/{x}.png"
)


In [ ]:
from tqdm import tqdm
import cv2

for idx, row in tqdm(df_clean.iterrows(), total=len(df_clean)):
    crop_path = row["crop_path"]
    ann = int(row["annotationID"])

    crop = cv2.imread(crop_path)
    if crop is None:
        continue

    mask = get_sam_mask(crop)
    mask_path = f"{masks_dir}/{ann}.png"
    cv2.imwrite(mask_path, mask)


  0%|          | 0/62614 [00:00<?, ?it/s]


NameError: name 'predictor' is not defined

In [ ]:
from mobile_sam import sam_model_registry, SamPredictor
import torch

checkpoint_path = "/content/mobile_sam.pt"

sam = sam_model_registry["vit_t"](checkpoint=checkpoint_path)
sam.to("cuda")
sam.half()   # FP16 for speed
predictor = SamPredictor(sam)


ModuleNotFoundError: No module named 'mobile_sam'

In [ ]:
import cv2
import numpy as np
import os
from tqdm import tqdm
from mobile_sam import sam_model_registry, SamPredictor
import torch

# Load MobileSAM (very fast)
sam = sam_model_registry["vit_t"](checkpoint="mobile_sam.pt")
sam.to("cuda")
sam.half()  # fp16 for extra speed

predictor = SamPredictor(sam)

def get_mask(crop):
    crop_rgb = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
    predictor.set_image(crop_rgb)

    input_point = np.array([[64, 64]])
    input_label = np.array([1])

    masks, _, _ = predictor.predict(
        point_coords=input_point,
        point_labels=input_label,
        multimask_output=False
    )

    return (masks[0] * 255).astype(np.uint8)


ModuleNotFoundError: No module named 'mobile_sam'

In [ ]:
from tqdm import tqdm
import cv2

for idx, row in tqdm(df_clean.iterrows(), total=len(df_clean)):
    crop_path = row["crop_path"]

    crop = cv2.imread(crop_path)
    if crop is None:
        print("Could not read:", crop_path)
        continue

    mask = get_sam_mask(crop)
    mask_path = f"{masks_dir}/{row['annotationID']}.png"
    cv2.imwrite(mask_path, mask)


  0%|          | 0/62614 [00:00<?, ?it/s]


NameError: name 'predictor' is not defined

In [ ]:
!pip install git+https://github.com/ChaoningZhang/MobileSAM.git


  Cloning https://github.com/ChaoningZhang/MobileSAM.git to /tmp/pip-req-build-onvazdp3
  Running command git clone --filter=blob:none --quiet https://github.com/ChaoningZhang/MobileSAM.git /tmp/pip-req-build-onvazdp3
  Resolved https://github.com/ChaoningZhang/MobileSAM.git to commit 34bbbfdface3c18e5221aa7de6032d7220c6c6a1
  Preparing metadata (setup.py) ... done
  Created wheel for mobile_sam: filename=mobile_sam-1.0-py3-none-any.whl size=42431 sha256=68e80e006bf089dbd81365a685feee7f810ea32ff7e9d6f58828ccc203764b7c
  Stored in directory: /tmp/pip-ephem-wheel-cache-zfms5jd2/wheels/5f/88/d6/5c0b5d4d64a06e19190d50269d8725c8aeadb128c966801af5
Successfully built mobile_sam


In [ ]:
!wget -O mobile_sam.pt https://github.com/ChaoningZhang/MobileSAM/releases/download/v1.0/mobile_sam.pt


--2025-11-30 19:55:18--  https://github.com/ChaoningZhang/MobileSAM/releases/download/v1.0/mobile_sam.pt
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-11-30 19:55:18 ERROR 404: Not Found.



In [ ]:
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install opencv-python matplotlib


  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-5001plrj
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-5001plrj
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py) ... done
  Created wheel for segment_anything: filename=segment_anything-1.0-py3-none-any.whl size=36592 sha256=85db61d8e0137c4f0c878a8cf326ba57ae8ae687f0f0c29f8932fe8a45c9faef
  Stored in directory: /tmp/pip-ephem-wheel-cache-1f0on599/wheels/29/82/ff/04e2be9805a1cb48bec0b85b5a6da6b63f647645750a0e42d4
Successfully built segment_anything


In [ ]:
!wget -O sam_vit_b.pth https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth


--2025-11-30 19:54:31--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.35.37.84, 13.35.37.90, 13.35.37.111, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.35.37.84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 375042383 (358M) [binary/octet-stream]
Saving to: ‘sam_vit_b.pth’

sam_vit_b.pth       100%[===================>] 357.67M   306MB/s    in 1.2s    

2025-11-30 19:54:32 (306 MB/s) - ‘sam_vit_b.pth’ saved [375042383/375042383]



In [ ]:
!ls -lh sam_vit_b.pth


-rw-r--r-- 1 root root 358M Apr  4  2023 sam_vit_b.pth


In [ ]:
import torch
from segment_anything import sam_model_registry, SamPredictor

device = "cuda"

sam_checkpoint = "sam_vit_b.pth"
model_type = "vit_b"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device)

predictor = SamPredictor(sam)


In [ ]:
import numpy as np
import cv2

def get_sam_mask(crop):
    crop_rgb = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
    predictor.set_image(crop_rgb)

    input_point = np.array([[64, 64]])
    input_label = np.array([1])

    masks, scores, _ = predictor.predict(
        point_coords=input_point,
        point_labels=input_label,
        multimask_output=False
    )

    mask = (masks[0] * 255).astype(np.uint8)
    return mask


In [ ]:
df_clean["crop_path"] = df_clean["annotationID"].apply(
    lambda x: f"{crops_dir}/{x}.png"
)


In [ ]:
from tqdm import tqdm

for idx, row in tqdm(df_clean.iterrows(), total=len(df_clean)):
    crop_path = row["crop_path"]
    ann = int(row["annotationID"])

    crop = cv2.imread(crop_path)
    if crop is None:
        print("Failed to read:", crop_path)
        continue

    mask = get_sam_mask(crop)
    cv2.imwrite(f"{masks_dir}/{ann}.png", mask)


  2%|▏         | 1559/62614 [10:59<6:11:19,  2.74it/s]

Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/630.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/630.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/631.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/631.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/632.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/632.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/633.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/633.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/633.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/634.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/634.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/635.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/635.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/635.png
Failed to read: /content/drive/MyD

  3%|▎         | 1764/62614 [11:53<4:39:19,  3.63it/s]

Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/712.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/712.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/713.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/713.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/714.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/714.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/715.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/715.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/716.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/716.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/717.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/717.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/718.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/718.png
Failed to read: /content/drive/MyD

 70%|███████   | 44134/62614 [5:14:36<1:27:55,  3.50it/s]

Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/18790.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/18790.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/18791.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/18791.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/18792.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/18792.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/18792.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/18793.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/18793.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/18794.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/18794.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/18795.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/18795.png
Failed to read: /content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/18796.png
Failed

100%|██████████| 62614/62614 [7:24:37<00:00,  2.35it/s]


In [5]:
images_dir = "/content/drive/MyDrive/MIDOGPP/MIDOGpp/images"
crops_dir  = "/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops"
masks_dir = "/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks"

In [6]:
import os
import pandas as pd

images_dir = "/content/drive/MyDrive/MIDOGPP/MIDOGpp/images"
crops_dir  = "/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops"
masks_dir  = "/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks"

def list_files_with_exts(path, exts=(".png", ".jpg", ".jpeg", ".tif", ".tiff")):
    return sorted([
        f for f in os.listdir(path)
        if f.lower().endswith(exts)
    ])

image_files = list_files_with_exts(images_dir)
crop_files  = list_files_with_exts(crops_dir)
mask_files  = list_files_with_exts(masks_dir)

df = pd.DataFrame({
    "annotation_id": [os.path.splitext(f)[0] for f in image_files],
    "image_path": [os.path.join(images_dir, f) for f in image_files]
})

# Add crop path only if exists
df["crop_path"] = df["annotation_id"].apply(
    lambda x: os.path.join(crops_dir, x + ".png") if (x + ".png") in crop_files
              else os.path.join(crops_dir, x + ".jpg") if (x + ".jpg") in crop_files
              else None
)

# Add mask path only if exists
df["mask_path"] = df["annotation_id"].apply(
    lambda x: os.path.join(masks_dir, x + ".png") if (x + ".png") in mask_files
              else os.path.join(masks_dir, x + ".jpg") if (x + ".jpg") in mask_files
              else None
)

# Drop rows missing ANY required component
df_clean = df.dropna(subset=["crop_path", "mask_path"]).reset_index(drop=True)

print("Total images:", len(image_files))
print("Usable complete samples:", len(df_clean))

df_clean.head()

Total images: 500
Usable complete samples: 403


,annotation_id,image_path,crop_path,mask_path
0,100,/content/drive/MyDrive/MIDOGPP/MIDOGpp/images/...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/1...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks/1...
1,101,/content/drive/MyDrive/MIDOGPP/MIDOGpp/images/...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/1...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks/1...
2,102,/content/drive/MyDrive/MIDOGPP/MIDOGpp/images/...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/1...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks/1...
3,103,/content/drive/MyDrive/MIDOGPP/MIDOGpp/images/...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/1...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks/1...
4,104,/content/drive/MyDrive/MIDOGPP/MIDOGpp/images/...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/1...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks/1...


In [7]:
import tensorflow as tf
IMG_SIZE = (256, 256)
BATCH_SIZE = 8
AUTOTUNE = tf.data.AUTOTUNE

def load_and_resize(path, channels):
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img, channels=channels, expand_animations=False)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, IMG_SIZE)
    return img

def load_fusion(image_path, crop_path, mask_path):
    image = load_and_resize(image_path, 3)
    crop  = load_and_resize(crop_path, 3)
    mask  = load_and_resize(mask_path, 1)
    mask  = tf.where(mask > 0.5, 1.0, 0.0)

    fused = tf.concat([image, crop, mask], axis=-1)    # (H, W, 7)
    return fused, mask

def build_dataset(df, batch_size=BATCH_SIZE, shuffle=False):
    ds = tf.data.Dataset.from_tensor_slices(
        (df["image_path"], df["crop_path"], df["mask_path"])
    )
    if shuffle:
        ds = ds.shuffle(buffer_size=len(df))

    ds = ds.map(lambda a,b,c: load_fusion(a,b,c),
                num_parallel_calls=AUTOTUNE)
    ds = ds.batch(batch_size).prefetch(AUTOTUNE)
    return ds

# Create splits
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df_clean, test_size=0.15, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)

train_ds = build_dataset(train_df, shuffle=True)
val_ds   = build_dataset(val_df)
test_ds  = build_dataset(test_df)

len(train_df), len(val_df), len(test_df)

(307, 35, 61)

In [8]:
import os
import pandas as pd

images_dir = "/content/drive/MyDrive/MIDOGPP/MIDOGpp/images"
crops_dir  = "/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops"
masks_dir  = "/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks"

def list_files(path):
    return sorted([f for f in os.listdir(path) if not f.startswith(".")])

crop_files = list_files(crops_dir)
mask_files = list_files(masks_dir)
image_files = list_files(images_dir)

# Extract annotation_id from crop files (remove extension)
anno_ids = [os.path.splitext(f)[0] for f in crop_files]

rows = []

for anno in anno_ids:
    crop_path = os.path.join(crops_dir, anno + ".png")
    if not os.path.exists(crop_path):
        crop_path = os.path.join(crops_dir, anno + ".jpg")
        if not os.path.exists(crop_path):
            continue

    mask_path = os.path.join(masks_dir, anno + ".png")
    if not os.path.exists(mask_path):
        mask_path = os.path.join(masks_dir, anno + ".jpg")
        if not os.path.exists(mask_path):
            continue

    # determine parent image
    # Usually annotation filename contains original patch/image ID
    image_id = anno.split("_")[0]   # IMPORTANT: adjust if needed

    # find matching image file
    possible_imgs = [
        os.path.join(images_dir, image_id + ext)
        for ext in [".png", ".jpg", ".jpeg", ".tif", ".tiff"]
    ]

    image_path = None
    for p in possible_imgs:
        if os.path.exists(p):
            image_path = p
            break

    if image_path is None:
        continue

    rows.append([anno, image_path, crop_path, mask_path])

df = pd.DataFrame(rows, columns=["annotation_id", "image_path", "crop_path", "mask_path"])

print("Total annotations:", len(anno_ids))
print("Usable complete samples:", len(df))
df.head()

Total annotations: 26129
Usable complete samples: 403


,annotation_id,image_path,crop_path,mask_path
0,100,/content/drive/MyDrive/MIDOGPP/MIDOGpp/images/...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/1...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks/1...
1,101,/content/drive/MyDrive/MIDOGPP/MIDOGpp/images/...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/1...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks/1...
2,102,/content/drive/MyDrive/MIDOGPP/MIDOGpp/images/...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/1...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks/1...
3,103,/content/drive/MyDrive/MIDOGPP/MIDOGpp/images/...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/1...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks/1...
4,104,/content/drive/MyDrive/MIDOGPP/MIDOGpp/images/...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/crops/1...,/content/drive/MyDrive/MIDOGPP/MIDOGpp/masks/1...


In [44]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 128   # A100 can handle this easily

In [45]:
@tf.function
def load_fused(crop_path, mask_path, label):
    crop = tf.io.read_file(crop_path)
    crop = tf.image.decode_png(crop, channels=3)
    crop = tf.image.resize(crop, IMG_SIZE)
    crop = tf.image.convert_image_dtype(crop, tf.float32)

    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)
    mask = tf.image.resize(mask, IMG_SIZE)
    mask = tf.image.convert_image_dtype(mask, tf.float32)

    fused = tf.concat([crop, mask], axis=-1)
    return fused, tf.cast(label, tf.float32)

In [46]:
def make_fast_dataset(df, shuffle=False, augment=False):
    crops  = tf.constant(df["crop_path"].tolist())
    masks  = tf.constant(df["mask_path"].tolist())
    labels = tf.constant(df["class"].astype("float32").tolist())

    ds = tf.data.Dataset.from_tensor_slices((crops, masks, labels))

    if shuffle:
        ds = ds.shuffle(len(df), seed=42)

    ds = ds.map(load_fused, num_parallel_calls=tf.data.AUTOTUNE)

    if augment:
        ds = ds.map(augment_fast, num_parallel_calls=tf.data.AUTOTUNE)

    ds = ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return ds

In [47]:
for X, y in train_ds.take(1):
    print(X.shape)
    break

(32, 224, 224, 4)


In [49]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

In [55]:
!watch -n 2 nvidia-smi

>

In [59]:
!rm -f /content/cache_mitotic.tfcache*

In [60]:
train_ds = make_dataset_from_crops(train_crops_tf, train_labels_tf, shuffle=True, augment=True)
val_ds   = make_dataset_from_crops(val_crops_tf, val_labels_tf)
test_ds  = make_dataset_from_crops(test_crops_tf, test_labels_tf)

In [63]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=callbacks
)

Epoch 1/50
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 12s/step - accuracy: 0.4285 - loss: -43.9185 
Epoch 1: val_accuracy improved from -inf to 0.44243, saving model to /content/drive/MyDrive/MIDOGPP/MIDOGpp/models/mobilenet_a100_best.h5


166/166 ━━━━━━━━━━━━━━━━━━━━ 1952s 12s/step - accuracy: 0.4285 - loss: -44.2192 - val_accuracy: 0.4424 - val_loss: -509.3212 - learning_rate: 1.0000e-04
Epoch 2/50
165/166 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.4286 - loss: -319.3235
Epoch 2: val_accuracy did not improve from 0.44243
166/166 ━━━━━━━━━━━━━━━━━━━━ 13s 78ms/step - accuracy: 0.4285 - loss: -320.5086 - val_accuracy: 0.4424 - val_loss: -919.9971 - learning_rate: 1.0000e-04
Epoch 3/50
165/166 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.4286 - loss: -737.7177
Epoch 3: val_accuracy did not improve from 0.44243
166/166 ━━━━━━━━━━━━━━━━━━━━ 13s 80ms/step - accuracy: 0.4285 - loss: -739.2674 - val_accuracy: 0.4424 - val_loss: -1298.5485 - learning_rate: 1.0000e-04
Epoch 4/50
165/166 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.4286 - loss: -1264.2596
Epoch 4: val_accuracy did not improve from 0.44243
166/166 ━━━━━━━━━━━━━━━━━━━━ 13s 77ms/step - accuracy: 0.4285 - loss: -1266.2213 - val_accuracy: 0.4424 - val_loss: 

In [24]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 128
AUTO = tf.data.AUTOTUNE

def mask_from_crop(crop_path):
    return tf.strings.regex_replace(crop_path, "/crops/", "/masks/")

@tf.function
def load_fused(crop_path, label):
    mask_path = mask_from_crop(crop_path)

    crop = tf.io.read_file(crop_path)
    crop = tf.image.decode_png(crop, channels=3)
    crop = tf.image.convert_image_dtype(crop, tf.float32)
    crop = tf.image.resize(crop, IMG_SIZE)

    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)
    mask = tf.image.convert_image_dtype(mask, tf.float32)
    mask = tf.image.resize(mask, IMG_SIZE)

    fused = tf.concat([crop, mask], axis=-1)
    return fused, label

def make_dataset(paths, labels):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(load_fused, num_parallel_calls=AUTO)
    ds = ds.batch(BATCH_SIZE).prefetch(AUTO)
    return ds

test_ds = make_dataset(test_paths, test_labels)

In [26]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_mobilenet():
    inp = layers.Input(shape=(224,224,4))

    # Split channels: RGB (3) + mask (1)
    rgb = inp[..., :3]
    mask = inp[..., 3:]

    # MobileNetV2 backbone
    base = tf.keras.applications.MobileNetV2(
        input_shape=(224,224,3),
        include_top=False,
        weights="imagenet"
    )
    x = base(rgb)
    x = layers.GlobalAveragePooling2D()(x)

    # Mask branch (simple CNN)
    m = layers.Conv2D(16, 3, activation="relu", padding="same")(mask)
    m = layers.GlobalAveragePooling2D()(m)

    # Fusion
    z = layers.Concatenate()([x, m])
    z = layers.Dense(128, activation="relu")(z)
    z = layers.Dropout(0.3)(z)
    out = layers.Dense(1, activation="sigmoid")(z)

    return models.Model(inp, out)


In [27]:
model = build_mobilenet()
# OR build_efficientnet()
# OR build_resnet()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [28]:
ckpt_path = "/content/drive/MyDrive/MIDOGPP/MIDOGpp/models/mobilenet_best.h5"
model.load_weights(ckpt_path)
print("Loaded best model weights!")

Loaded best model weights!


In [30]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [31]:
loss, acc = model.evaluate(test_ds)
print("Test Loss:", loss)
print("Test Accuracy:", acc)

21/21 ━━━━━━━━━━━━━━━━━━━━ 110s 4s/step - accuracy: 0.7941 - loss: 1.5249
Test Loss: 1.563114881515503
Test Accuracy: 0.7872177362442017
